# AdVantage
adVantage is a collection of tools to be able to detect moving aircraft in a video from the vantage platform. Below is a walkthrough how to perform the tracking process.

https://vantage.earthi.world/explore/

### First install requirements

In [1]:
%cd ~/AdVantage
!python3 -m pip install -r requirements.txt

/home/jovyan/AdVantage


### Download yolov5 repository

Yolov5 repository needs to be installed before the AdVantage example will work. You will need to clone the repository into the current working directory.

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 11290, done.
remote: Total 11290 (delta 0), reused 0 (delta 0), pack-reused 11290
Receiving objects: 100% (11290/11290), 11.15 MiB | 12.61 MiB/s, done.
Resolving deltas: 100% (7812/7812), done.


In [3]:
%cd ~/AdVantage/yolov5

/home/jovyan/AdVantage/yolov5


In [4]:
# download weights
%cd ~/AdVantage
!mkdir input
%cd ~/AdVantage/input
!gdown "https://drive.google.com/uc?id=1GBJyQqSQaUJdeXBS-CzZwjzEeAUrv47-"

%cd ~/AdVantage
!mkdir output

/home/jovyan/AdVantage
/home/jovyan/AdVantage/input
Downloading...
From: https://drive.google.com/uc?id=1GBJyQqSQaUJdeXBS-CzZwjzEeAUrv47-
To: /home/jovyan/AdVantage/input/best.pt
100%|███████████████████████████████████████| 14.5M/14.5M [00:00<00:00, 182MB/s]
/home/jovyan/AdVantage


### Authenticate with the Vantage platform
To interact with the api you will need an API key that can be generated on the vantage platform. If you do not have this please generate a key from your account area.
https://vantage.earthi.world/advanced/#/account . Your username will be the email address you used to sign up with

In [7]:
%cd ~/AdVantage
import getpass
from vantage_api.services import VantageServiceApi

user = input('Vantage Email Address: ')
apiKey = getpass.getpass('API Key:')

vantageApi = VantageServiceApi(user, apiKey)

/Users/jw/AdVantage


KeyboardInterrupt: Interrupted by user

### Searching for an Airport
You are able to search for airports using the airport database. If you know the IATA code you can use the code below to find the airport. The result will give to longitude and latiute information you can use to search for videos on the Vantage platform. A couple of codes that can be used here:

* AKL

In [12]:
from airport_database.airports import Airports
from vantage_api.utils import GeoRequest

iataCode = input('IATA Code: ')
airports = Airports()
airport = airports.findByIATACode(iataCode.upper())
print('Airport: '+airport.name)
print('Latitude: '+str(airport.lat_decimal))
print('Longitude: '+str(airport.lon_decimal))

geoData = GeoRequest()
geoData.fromBoundingBox(airport.boundingBox(halfSideInKm=10))

features = vantageApi.getSearch(geoData).getFeatures()
for key in features.keys():
    print(key)

chosenFeature = None
while chosenFeature == None:
    chosenKey = input('Enter a Key to Process: ')    
    if chosenKey in features.keys():
        chosenFeature = features[chosenKey]        

print('Chosen: '+chosenFeature)

IATA Code:  AKL
Airport: AUCKLAND INTERNATIONAL
Latitude: -37.008
Longitude: 174.792
VX020001c0
Enter a Key to Process:  AUCKLAND INTERNATIONAL
Enter a Key to Process:  Airport
Enter a Key to Process:  VX020001c0
Chosen: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020001c0.zip


### Download Video From Vantage Platform
Downloading the image from the vantage platform requires you to create a job and wait for its result. The API wrapper on AdVantage will allow you to do this in one command.

You can also download a stablised video using the `stabliseVideo` service.

Please choose one of two pieces of code to start the download. 

**Please note it can take quite some time to download these videos from the Vantage platform**

In [ ]:
outputFiles = vantageApi.downloadFiles(chosenFeature,'./input', verbose=True)
print(outputFiles)

In [ ]:
%cd ~/AdVantage
!mkdir input
outputFiles = vantageApi.stabliseVideo(chosenFeature,'./input', verbose=True)
print(outputFiles)

current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: PENDING
current job status: 

### Download Video From Vantage Platform
Often the vantage platform may not be responsive, instead we can download an example stabilised video from drive

Users also have the option of supplying their own video outside of the vantage platform

In [5]:
%cd ~/AdVantage/input
!gdown "https://drive.google.com/uc?id=1xb83jYUIey1MFqC_yGzEIleE_055ExG_"

/home/jovyan/AdVantage/input
Downloading...
From: https://drive.google.com/uc?id=1xb83jYUIey1MFqC_yGzEIleE_055ExG_
To: /home/jovyan/AdVantage/input/VX020001c0.zip
100%|███████████████████████████████████████| 1.51G/1.51G [00:06<00:00, 227MB/s]


In [6]:
%cd ~/AdVantage/input
!unzip VX020001c0.zip -d ./

/home/jovyan/AdVantage/input
Archive:  VX020001c0.zip
 extracting: ./VX020001c0_VidMD.xml  
 extracting: ./VX020001c0.mp4        
 extracting: ./VX020001c0_AcqMD.xml  
 extracting: ./VX020001c0_geometry.xml  
 extracting: ./VX020001c0_Thumb.jpg  
 extracting: ./VX020001c0_Catalogue.xml  
 extracting: ./VX020001c0_Browse.mp4  


In [1]:
chosenFeature = './input/VX020001c0.mp4'

### Detect Aircraft Movements
The AdVantage platform is written to run via a pipeline. This allows you to easily create tasks for each frame of the video and perform tasks

In [ ]:
%cd ~/AdVantage
import os
import zipfile
import sys
sys.path.insert(0, './yolov5')

from advantage.app import AdVantage
from advantage.handlers import *
from advantage.handlers.objectracker import ObjectTracker
from advantage.handlers.verbose import Verbose
from advantage.handlers.groundcontrolpoints import GroundControlPoints
from advantage.handlers.backgroundframe import BackgroundFrame
from advantage.handlers.stablisationdectection import StablisationDetection
from advantage.handlers.videowriter import VideoWriter
from advantage.handlers.yolo import YoloProcessor
from advantage.handlers.movementfilter import MovementFilter
from advantage.handlers.visulisation import VideoPredictionVisualisation
from advantage.handlers.pipelinekiller import PipelineKiller
from advantage.handlers.geodata import VideoAttachGeoData
from advantage.handlers.geotracker import GeoObjectTracker
from advantage.trackers import ObjectDetectionTracker

/home/jovyan/AdVantage


In [8]:
%cd ~/AdVantage
app = AdVantage()
cwd = '.'
outputDir = os.path.join(cwd, 'output')

if not os.path.exists(outputDir):
    os.makedirs(outputDir)
   
filename, file_extension = os.path.splitext(chosenFeature)
filename = os.path.basename(filename)

pipeline = app.pipeline_factory([
    Verbose(), #Prints information to console
    PipelineKiller(frames_to_process = 10), #kills process after x frames
    #GroundControlPoints(), #hard coded for VX020001c0 frame 1
    BackgroundFrame(scale=90), #Creates a resized frame to process on
    StablisationDetection(bbox_size=100),
    VideoWriter(
        os.path.join(cwd,'output',filename+'.mp4'), #Video Path to Save to if set to true
        output_video=False, #Output the video
        image_frame_output_dir=os.path.join(cwd,'output') #Outputs Image of each frame
    ),
    YoloProcessor(
        os.path.join(cwd,'input/best.pt'), #weights file. must be absolute
        conf_thres=0.8, #only save predictions over % 0 to 1,
        skip_frames=10,
        first_frame_only=True
    ), 
    ObjectTracker(ObjectDetectionTracker(offset_allowance=10), isolateObjectIds=[9], sanity_lines=True),
    MovementFilter(),
    #VideoAttachGeoData('input/VX020001c0_geometry.xml'), #Attach geo data to frame
    #GeoObjectTracker(),
    VideoPredictionVisualisation(include=['frame_objects','stablisation_points']), # Applies details to video/image frames
])

#if isZip:
#    pipelines.append(VideoAttachGeoData(os.path.join(cwd,'input',filename+'_geometry.xml')))

result = app.process_video(os.path.join(cwd,'input',filename+'.mp4'), pipeline)
app.save_output_to_json(result, os.path.join(cwd,'output',filename+'.json'))

YOLOv5 🚀 v6.1-32-gc13d4ce torch 1.10.0+cu102 CPU



/home/jovyan/AdVantage


Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


video properties are:
fps:  10.0
frame_width:  3072
frame_height:  3072
Processing Frame: 0
Processing Frame: 1
this long lat 174.79215365641488 -37.00716636141478
last_long_lat [174.79215365641488, -37.00716636141478]
last_velocity 0
forward_azimuth, backward_azimuth, distance 0.0 180.0 0.0
velocity 0.0
acceleration 0.0
this long lat 174.79279531604783 -37.006067488211045
last_long_lat [174.79279531604783, -37.006067488211045]
last_velocity 0
forward_azimuth, backward_azimuth, distance 0.0 180.0 0.0
velocity 0.0
acceleration 0.0
this long lat 174.78663239910816 -37.00806435352822
last_long_lat [174.78663239910816, -37.00806435352822]
last_velocity 0
forward_azimuth, backward_azimuth, distance 0.0 180.0 0.0
velocity 0.0
acceleration 0.0
this long lat 174.8049122374886 -37.00209723329974
last_long_lat [174.8049122374886, -37.00209723329974]
last_velocity 0
forward_azimuth, backward_azimuth, distance 0.0 180.0 0.0
velocity 0.0
acceleration 0.0
this long lat 174.79198951092738 -37.0090332

# Pipeline todo
- get video from vantage platform (or wget a pre-downloaded video for POC)
- split into frames  or load video frame per frame using open cv
- run yolo on each frame recording bbox of each image in yolo format
- convert yolo format bbox to geographic format using frame metadata and projection
    - display on map for POC
- create binary image which is geo based
- run particle filter or something similar using CV2 to track objects
- use tracked objects positioning etc. to calculate speed and heading
- display this both in a map and on the original video